Proceso automatizacion Exito

In [1]:
pip install Playwright pandas google-cloud-bigquery pyarrow pandas-gbq


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementClickInterceptedException
from google.oauth2 import service_account
import time
import datetime
import os
import glob
import pandas as pd
import re
import shutil
from google.cloud import storage


In [3]:
download_path = r"C:\Users\Lenovo\Downloads\scripts\Exito\Exito" #Cambiar por la ruta donde se quieran descargar los archivos

In [4]:
archivos_excel = glob.glob(os.path.join(download_path, "*.xlsx")) + glob.glob(os.path.join(download_path, "*.xls"))

# Eliminar cada archivo encontrado
for archivo in archivos_excel:
    try:
        os.remove(archivo)
        print(f"Archivo eliminado: {archivo}")
    except Exception as e:
        print(f"No se pudo eliminar {archivo}: {e}")

Archivo eliminado: C:\Users\Lenovo\Downloads\scripts\Exito\Exito\Qlik Sense - Reporte en línea - 23 de agosto de 2025.xlsx


In [5]:
def setup_driver():
    """Configura y retorna el driver de Chrome"""
    chrome_options = Options()
    prefs = {
    "download.default_directory": download_path,  # Carpeta de descargas
    "download.prompt_for_download": False,        # No preguntar
    "download.directory_upgrade": True,           # Forzar uso de carpeta
    "safebrowsing.enabled": True                  # Evitar bloqueos
    }
    chrome_options.add_experimental_option("prefs", prefs)

# Iniciar driver con configuración
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
    )

    return driver

def wait_captcha():
    """Espera para resolver CAPTCHA manualmente"""
    print("⏰ Resuelve el CAPTCHA manualmente (15 segundos)...")
    for i in range(20, 0, -1):
        print(f"Tiempo: {i}s", end="\r")
        time.sleep(1)
    print("\n✅ Continuando...")

def login_to_site(email, password):
    """
    Función principal para hacer login automático
    Retorna el driver listo para usar
    """
    print("🚀 INICIANDO LOGIN AUTOMÁTICO")
    print("=" * 40)
    
    # Configurar driver
    driver = setup_driver()
    
    try:
        # 1. Navegar a la página
        print("📍 Navegando a la página...")
        driver.get("https://prescriptivalatam.com/")
        
        # 2. Llenar credenciales
        print("👤 Llenando credenciales...")
        email_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "email"))
        )
        password_field = driver.find_element(By.ID, "password")
        
        email_field.clear()
        email_field.send_keys(email)
        password_field.clear()
        password_field.send_keys(password)
        
        # 3. Manejar reCAPTCHA
        print("🤖 Clickeando CAPTCHA...")
        recaptcha_iframe = WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe[contains(@src, 'recaptcha')]"))
        )
        checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "recaptcha-checkbox-border"))
        )
        checkbox.click()
        driver.switch_to.default_content()
        
        # 4. Esperar resolución manual del CAPTCHA
        wait_captcha()
        
        # 5. Hacer login
        print("🔐 Ejecutando login...")
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
        )
        login_button.click()
        time.sleep(3)
        
        # 6. Manejar modal si aparece
        print("📋 Verificando modal...")
        try:
            continuar_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "btn__applyFilter"))
            )
            continuar_btn.click()
            print("✅ Modal cerrado")
            time.sleep(2)
            
            # Segundo login después del modal
            login_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
            )
            login_button.click()
            time.sleep(2)
            
        except:
            print("ℹ️ No hay modal - continuando...")
        
        print("🎉 LOGIN COMPLETADO EXITOSAMENTE")
        print("=" * 40)
        print(f"📍 URL: {driver.current_url}")
        print("✅ Driver listo para scraping")
        
        return driver
        
    except Exception as e:
        print(f"❌ ERROR: {e}")
        driver.quit()
        return None

def click_cen_colaboracion(driver):
    """Hace clic en 'Cen Colaboración' del menú"""
    print("🎯 Clickeando 'Cen Colaboración'...")
    
    selectors = [
        (By.ID, "menu-options-1"),
        (By.XPATH, "//button[@aria-label='Cen Colaboración']"),
        (By.XPATH, "//button[.//img[@alt='Cen Colaboración']]")
    ]
    
    for selector_type, selector_value in selectors:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].scrollIntoView();", element)
            time.sleep(1)
            element.click()
            print("✅ Cen Colaboración activado")
            time.sleep(2)
            return True
        except:
            continue
    
    print("❌ No se pudo activar Cen Colaboración")
    return False

# ============================================================================
# EJECUCIÓN PRINCIPAL - CELDA PRINCIPAL
# ============================================================================

# Credenciales
EMAIL = "grandessuperficies@donchicharron.com.co"
PASSWORD = "Mar2cazaz123*"

# Ejecutar login
driver = login_to_site(EMAIL, PASSWORD)

if driver:
    # Activar Cen Colaboración
    click_cen_colaboracion(driver)
    print("\n🚀 SISTEMA LISTO PARA LAS SIGUIENTES CELDAS")
else:
    print("❌ Error en login - revisar credenciales o conexión")

🚀 INICIANDO LOGIN AUTOMÁTICO
📍 Navegando a la página...
👤 Llenando credenciales...
🤖 Clickeando CAPTCHA...
⏰ Resuelve el CAPTCHA manualmente (15 segundos)...
Tiempo: 1ss
✅ Continuando...
🔐 Ejecutando login...
📋 Verificando modal...
ℹ️ No hay modal - continuando...
🎉 LOGIN COMPLETADO EXITOSAMENTE
📍 URL: https://prescriptivalatam.com/user/home
✅ Driver listo para scraping
🎯 Clickeando 'Cen Colaboración'...
✅ Cen Colaboración activado

🚀 SISTEMA LISTO PARA LAS SIGUIENTES CELDAS


In [6]:
success = click_cen_colaboracion(driver)

🎯 Clickeando 'Cen Colaboración'...
✅ Cen Colaboración activado


In [7]:
try:
    # Múltiples selectores para encontrar el botón Indicadores
    Reporte_dinamico = [
        # Por texto exacto del botón
        (By.XPATH, "//button[text()='Reporte dinámico']"),
        # Por texto que contenga "Reporte dinámico"
        (By.XPATH, "//button[contains(text(), 'Reporte dinámico')]"),
        # Por texto que contenga "Reporte" (más flexible)
        (By.XPATH, "//button[contains(text(), 'Reporte')]"),
        # Por texto que contenga "dinámico"
        (By.XPATH, "//button[contains(text(), 'dinámico')]"),
        # Por clase específica del sidebar
        (By.XPATH, "//button[contains(@class, 'sideBar__button__options') and contains(text(), 'Reporte')]"),
        # Por combinación de clases Material-UI y texto
        (By.XPATH, "//button[contains(@class, 'MuiButton-root') and contains(text(), 'Reporte dinámico')]"),
        # Por tipo y texto
        (By.XPATH, "//button[@type='button' and contains(text(), 'Reporte dinámico')]"),
        # Buscar cualquier botón del sidebar que contenga "dinámico"
        (By.XPATH, "//button[contains(@class, 'sideBar__button__options') and contains(text(), 'dinámico')]"),
        # Por CSS y contenido
        (By.XPATH, "//button[@class='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium MuiButton-colorPrimary MuiButton-disableElevation MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium MuiButton-colorPrimary MuiButton-disableElevation sideBar__button__options css-1cy74wo' and contains(text(), 'Reporte')]")
    ]
    
    
    accordion_clicked = False
    
    for selector_type, selector_value in Reporte_dinamico:
        try:
            print(f"🔍 Intentando selector: {selector_value}")
            
            # Esperar a que el elemento sea clickeable
            accordion_element = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            
            print(f"✅ Accordion encontrado: {accordion_element.tag_name}")
            print(f"📋 Clases: {accordion_element.get_attribute('class')}")
            
            # Hacer scroll al elemento
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", accordion_element)
            time.sleep(2)
            
            # Verificar si ya está expandido antes de hacer click
            classes = accordion_element.get_attribute('class')
            is_expanded = 'Mui-expanded' in classes if classes else False
            
            if is_expanded:
                print("ℹ️ El accordion ya está expandido, pero haciendo click de todas formas...")
            else:
                print("📂 El accordion está contraído, expandiendo...")
            
            # Intentar click normal
            try:
                accordion_element.click()
                print("✅ Click normal exitoso en accordion")
                accordion_clicked = True
                break
            except Exception as click_error:
                print(f"🔄 Click normal falló: {str(click_error)[:50]}...")
                print("🔄 Intentando con JavaScript...")
                
                # Si el click normal falla, usar JavaScript
                driver.execute_script("arguments[0].click();", accordion_element)
                print("✅ Click con JavaScript exitoso en accordion")
                accordion_clicked = True
                break
                
        except Exception as e:
            print(f"❌ Selector falló: {str(e)[:60]}...")
            continue
    
    if accordion_clicked:
        print("🎉 Accordion clickeado exitosamente")
        
        # Esperar a que se expanda/contraiga
        print("⏳ Esperando animación del accordion...")
        time.sleep(3)
        
        # Verificar el estado final
        try:
            # Buscar el accordion nuevamente para verificar estado
            final_accordion = driver.find_element(By.CLASS_NAME, "MuiAccordionSummary-expandIconWrapper")
            final_classes = final_accordion.get_attribute('class')
            is_expanded = 'Mui-expanded' in final_classes if final_classes else False
            
            if is_expanded:
                print("✅ Accordion está EXPANDIDO")
            else:
                print("📁 Accordion está CONTRAÍDO")
                
        except:
            print("⚠️ No se pudo verificar el estado final del accordion")

except Exception as e:
    print(f"❌ Error crítico al intentar hacer click en accordion: {e}")
    print("🔍 Verificando estado del driver...")
    try:
        print(f"📍 URL actual: {driver.current_url}")
        print(f"📄 Título: {driver.title}")
    except:
        print("❌ Driver no está disponible")


🔍 Intentando selector: //button[text()='Reporte dinámico']
✅ Accordion encontrado: button
📋 Clases: MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium MuiButton-colorPrimary MuiButton-disableElevation MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium MuiButton-colorPrimary MuiButton-disableElevation sideBar__button__options css-1cy74wo
📂 El accordion está contraído, expandiendo...
✅ Click normal exitoso en accordion
🎉 Accordion clickeado exitosamente
⏳ Esperando animación del accordion...
📁 Accordion está CONTRAÍDO


In [8]:
def click_reporte_en_linea(driver):
    """
    Versión simplificada que se detiene después del primer click exitoso
    """
    import time
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
    
    print("🎯 Intentando hacer clic en 'Reporte en línea'...")
    
    selectors = [
        (By.XPATH, "//button[text()='Reporte en línea']"),
        (By.XPATH, "//button[contains(text(), 'Reporte en línea')]"),
        (By.XPATH, "//button[contains(@class, 'MuiButton-root') and text()='Reporte en línea']"),
    ]
    
    wait = WebDriverWait(driver, 15)
    current_url = driver.current_url  # Guardar URL actual
    
    for i, (selector_type, selector_value) in enumerate(selectors, 1):
        try:
            print(f"🔍 Estrategia {i}: {selector_value[:50]}...")
            
            element = wait.until(EC.element_to_be_clickable((selector_type, selector_value)))
            print(f"✅ Elemento encontrado y clickeable")

            # Click y verificación inmediata
            element.click()
            print("✅ Click realizado")
            
            # Esperar un cambio (URL, elemento desaparecido, etc.)
            time.sleep(3)
            
            # Verificar si hubo cambio
            if driver.current_url != current_url:
                print("✅ URL cambió - Click exitoso confirmado")
                return True
            
            # O verificar si el elemento ya no existe/es stale
            try:
                element.is_enabled()  # Esto fallará si es stale
            except StaleElementReferenceException:
                print("✅ Elemento stale - Click exitoso confirmado")
                return True
                
            print("✅ Click procesado correctamente")
            return True
                
        except (TimeoutException, ElementClickInterceptedException, StaleElementReferenceException) as e:
            if isinstance(e, StaleElementReferenceException):
                print("✅ Stale element - Click anterior exitoso")
                return True
            print(f"❌ Estrategia {i} falló: {str(e)[:50]}...")
            continue
            
        except Exception as e:
            print(f"❌ Estrategia {i} falló: {str(e)[:50]}...")
            continue
    
    print("❌ Todas las estrategias fallaron")
    return False

In [9]:
# Guarda ventana original
ventana_original = driver.current_window_handle

In [10]:
click_reporte_en_linea(driver)

🎯 Intentando hacer clic en 'Reporte en línea'...
🔍 Estrategia 1: //button[text()='Reporte en línea']...
✅ Elemento encontrado y clickeable
✅ Click realizado
✅ Elemento stale - Click exitoso confirmado


True

In [11]:
# Cambia a la nueva ventana
driver.implicitly_wait(4)
todas_las_ventanas = driver.window_handles
for ventana in todas_las_ventanas:
    if ventana != ventana_original:
        driver.switch_to.window(ventana)
        break   

In [12]:
# Espera a que el botón de marcadores sea clickeable
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-tid="assets-button-bookmarks"]'))
)
element.click()

In [13]:
# Intenta hacer clic en el botón de la lista privada
try:
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-testid="privateList-list-item-button-7cf8658b-22b6-476c-8a67-aa90c13fc026"]'))
        )
    element.click()
except:
    print("No se encontró el botón de la lista privada")

In [14]:
# Acepta terminos
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.lui-button.ng-scope.ng-isolate-scope'))
)
element.click()

In [15]:
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'h6[aria-label="Mes"]'))
)
element.click()
time.sleep(2)

In [16]:
# Obtener el mes actual en español
meses = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

mes_actual = datetime.datetime.now().month
nombre_mes = meses[mes_actual]

# Hacer clic en el mes actual
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'//span[text()="{nombre_mes}"]'))
)
element.click()
print(f"Seleccionado mes: {nombre_mes}")
time.sleep(3)

Seleccionado mes: Septiembre


In [17]:
# Se sale de la ventana del mes
boton = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="actions-toolbar-confirm"]'))
)
boton.click()
time.sleep(2)

In [18]:
# Hacer clic en el año
elemento = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'h6[aria-label="Año"]'))
)
elemento.click()
time.sleep(3)

In [19]:
# Hacer clic en el año 2025
elemento = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//span[text()="2025"]'))
)
elemento.click()
time.sleep(3)

In [20]:
#  Se sale de la ventana del año
boton = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="actions-toolbar-confirm"]')
boton.click()
time.sleep(3)

In [21]:
# Esperar hasta que el elemento sea visible
elemento = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//div[text()="Reporte en línea"]'))
)

# Hacer clic derecho (context click)
acciones = ActionChains(driver)
acciones.context_click(elemento).perform()


In [22]:
# Esperar a que aparezca la opción del menú contextual "Descargar"
descargar_opcion = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//li[@id="download" or @tid="download" or .//span[text()="Descargar"]]'))
)

# Hacer clic en esa opción
descargar_opcion.click()
time.sleep(2)

In [23]:
boton_descarga_final = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="download-dialog-download-button"]'))
)

# Hacer clic en el botón
boton_descarga_final.click()

In [24]:
time.sleep(7) # Espera para asegurarse de que la descarga se inicie

In [25]:
cerrar_btn = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="download-dialog-close" or @tid="download-cancel"]'))
)
cerrar_btn.click()
print("✅ Modal de descarga cerrado")

✅ Modal de descarga cerrado


In [26]:
# Espera a que el botón de marcadores sea clickeable
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-tid="assets-button-bookmarks"]'))
)
element.click()

In [27]:
# Esperar y hacer clic en "Inventario1"
inventario_btn = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, '//span[text()="Inventario1"]'))
)
inventario_btn.click()
print("✅ Clic en Inventario1")


✅ Clic en Inventario1


In [28]:
# Acepta terminos
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.lui-button.ng-scope.ng-isolate-scope'))
)
element.click()

In [29]:
# Esperar y hacer clic en el botón de limpiar selección de month_name
clear_btn = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="clear-month_name"]'))
)
clear_btn.click()
print("✅ Clic en limpiar selección de month_name")

✅ Clic en limpiar selección de month_name


In [30]:
# Celda 1: Limpiar year_number y esperar estabilización
try:
    print("🔄 Limpiando selección de year_number...")
    year_clear_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="clear-year_number"]'))
    )
    year_clear_btn.click()
    print("✅ Clic en limpiar selección de year_number")
    
    # Esperar más tiempo para que se estabilice la interfaz
    print("⏳ Esperando estabilización de la interfaz...")
    time.sleep(3)
    
except Exception as e:
    print(f"❌ Error limpiando year_number: {e}")

# Celda 2: Múltiples estrategias para encontrar el selector de Mes
try:
    print("🔍 Buscando selector de Mes con múltiples estrategias...")
    
    # Lista de selectores a probar
    month_selectors = [
        # Selectores originales
        (By.CSS_SELECTOR, 'h6[aria-label="Mes"]'),
        
        # Por texto visible
        (By.XPATH, '//h6[text()="Mes"]'),
        (By.XPATH, '//h6[contains(text(), "Mes")]'),
        
        # Por clase y contenido
        (By.XPATH, '//h6[contains(@class, "MuiTypography") and contains(text(), "Mes")]'),
        
        # Buscar por el contenedor del filtro de Mes
        (By.XPATH, '//div[contains(@class, "filter") and .//text()[contains(., "Mes")]]//h6'),
        
        # Por data-testid si existe
        (By.CSS_SELECTOR, '[data-testid*="month"]'),
        (By.CSS_SELECTOR, '[data-testid*="mes"]'),
        
        # Buscar cualquier elemento clickeable que contenga "Mes"
        (By.XPATH, '//*[@role="button" and contains(text(), "Mes")]'),
        (By.XPATH, '//button[contains(text(), "Mes")]'),
        
        # Por posición en la interfaz (segundo elemento de filtro)
        (By.XPATH, '(//h6[contains(@aria-label, "")])[2]'),
        
        # Buscar en contenedores específicos de filtros
        (By.XPATH, '//div[contains(@class, "filter-container")]//h6[contains(text(), "Mes")]'),
        
        # Selector más genérico basado en la estructura
        (By.XPATH, '//h6[@aria-label and contains(@aria-label, "M")]'),
    ]
    
    element_found = False
    
    for i, (selector_type, selector_value) in enumerate(month_selectors, 1):
        try:
            print(f"🔍 Estrategia {i}: {selector_value}")
            
            # Esperar a que el elemento esté presente
            element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            
            # Verificar que esté visible y clickeable
            if element.is_displayed() and element.is_enabled():
                print(f"✅ Elemento encontrado con estrategia {i}")
                print(f"📋 Tag: {element.tag_name}")
                print(f"📋 Texto: '{element.text}'")
                print(f"📋 Aria-label: '{element.get_attribute('aria-label')}'")
                print(f"📋 Clases: {element.get_attribute('class')}")
                
                # Hacer scroll al elemento
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
                time.sleep(1)
                
                # Intentar hacer click
                try:
                    element.click()
                    print("✅ Click exitoso en selector de Mes")
                    element_found = True
                    break
                except Exception as click_error:
                    print(f"🔄 Click normal falló: {click_error}")
                    # Intentar con JavaScript
                    driver.execute_script("arguments[0].click();", element)
                    print("✅ Click con JavaScript exitoso")
                    element_found = True
                    break
                    
            else:
                print(f"⚠️ Elemento no visible/clickeable con estrategia {i}")
                
        except Exception as e:
            print(f"❌ Estrategia {i} falló: {str(e)[:50]}...")
            continue
    
    if not element_found:
        print("❌ No se encontró el selector de Mes con ninguna estrategia")
        
except Exception as e:
    print(f"❌ Error crítico buscando selector de Mes: {e}")

🔄 Limpiando selección de year_number...
✅ Clic en limpiar selección de year_number
⏳ Esperando estabilización de la interfaz...
🔍 Buscando selector de Mes con múltiples estrategias...
🔍 Estrategia 1: h6[aria-label="Mes"]
✅ Elemento encontrado con estrategia 1
📋 Tag: h6
📋 Texto: 'Mes'
📋 Aria-label: 'Mes'
📋 Clases: MuiTypography-root MuiTypography-subtitle2 MuiTypography-noWrap  css-en83of
✅ Click exitoso en selector de Mes


In [31]:
# Hacer clic en el mes actual
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'//span[text()="{nombre_mes}"]'))
)
element.click()

In [32]:
# Se sale de la ventana del mes
boton = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="actions-toolbar-confirm"]')
boton.click()

In [33]:
# Hacer clic en el año
elemento = driver.find_element(By.CSS_SELECTOR, 'h6[aria-label="Año"]')
elemento.click()


In [34]:
try:
    print("🔄 Intentando con XPath específico...")
    
    # XPath más específico para tu elemento
    wait = WebDriverWait(driver, 10)
    elemento = wait.until(EC.element_to_be_clickable((By.XPATH, 
        '//span[@class="njs-1ecc-Typography-root njs-1ecc-Typography-body2 njs-1ecc-Typography-alignRight RowColumn-labelText css-78lweu"]//span[text()="2025"]')))
    
    # Verificar que esté visible
    if elemento.is_displayed() and elemento.is_enabled():
        print("✅ Elemento encontrado y visible")
        
        # Hacer scroll
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", elemento)
        time.sleep(1)
        
        # Intentar click normal primero
        try:
            elemento.click()
            print("✅ Click normal exitoso")
            click_realizado = True
        except ElementClickInterceptedException:
            # Si falla, usar JavaScript
            driver.execute_script("arguments[0].click();", elemento)
            print("✅ Click JavaScript exitoso")
            click_realizado = True
            
    else:
        print("❌ Elemento no visible o no habilitado")
        click_realizado = False
        
except Exception as e:
    print(f"❌ Error en estrategia 1: {e}")
    click_realizado = False

🔄 Intentando con XPath específico...
✅ Elemento encontrado y visible
✅ Click normal exitoso


In [35]:
#  Se sale de la ventana del año
boton = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="actions-toolbar-confirm"]')
boton.click()

In [36]:
# Esperar hasta que el elemento sea visible
elemento = WebDriverWait(driver, 5).until(
    EC.visibility_of_element_located((By.XPATH, '//div[text()="Reporte en línea"]'))
)

# Hacer clic derecho (context click)
acciones = ActionChains(driver)
acciones.context_click(elemento).perform()

In [37]:
# Esperar a que aparezca la opción del menú contextual "Descargar"
descargar_opcion = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//li[@id="download" or @tid="download" or .//span[text()="Descargar"]]'))
)

# Hacer clic en esa opción
descargar_opcion.click()

In [38]:
boton_descarga_final = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="download-dialog-download-button"]'))
)

# Hacer clic en el botón descarga final
boton_descarga_final.click()
time.sleep(10)  # Espera para asegurarse de que la descarga se inicie
driver.quit()  # Cerrar el navegador al finalizar

In [39]:
# Identifica y renombra los archivos descargados, el primer archivo descargado es Ventas Mensuales y el segundo es Inventario
def renombrar_dos_archivos(): #
    # Esperar a que existan ambos archivos
    while True:
        archivos = glob.glob(os.path.join(download_path, "Qlik Sense - Reporte en línea*.xlsx"))
        archivos = [f for f in archivos if not f.endswith(".crdownload")]
        if len(archivos) >= 2:
            break
        time.sleep(1)  # espera 1 segundo y vuelve a verificar

    # Identificar el que tiene (1) → Inventario, y el que no → Ventas Mensuales
    for archivo in archivos:
        fecha = archivo.split("Qlik Sense - Reporte en línea - ")[1].replace(".xlsx", "").replace(" (1)", "")
        if "(1)" in archivo:
            nuevo_nombre = f"Qlik Sense - Reporte en línea - {fecha} Inventario.xlsx"
        else:
            nuevo_nombre = f"Qlik Sense - Reporte en línea - {fecha} Ventas Mensuales.xlsx"
        
        nuevo_path = os.path.join(download_path, nuevo_nombre)
        shutil.move(archivo, nuevo_path)
        print(f"✅ Renombrado: {os.path.basename(archivo)} → {os.path.basename(nuevo_path)}")
renombrar_dos_archivos()


KeyboardInterrupt: 

Subir datos al Bucket

In [ ]:
credentials_path = "credentials/croc-454221-e1a3c2e02181.json"
bucket_name = "bucket-quickstart_croc_830"

In [ ]:
def limpiar_bucket():
    """
    Función para eliminar archivos existentes en el bucket antes de subir nuevos
    """
    try:
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Rutas a limpiar
        rutas_a_limpiar = [
            "raw/Ventas/moderno/exito/inventario/",
            "raw/Ventas/moderno/exito/ventas/"
        ]
        
        print("🧹 Limpiando archivos existentes en el bucket...")
        
        for ruta in rutas_a_limpiar:
            # Listar archivos en la ruta
            blobs = list(bucket.list_blobs(prefix=ruta))
            archivos_xlsx = [blob for blob in blobs if blob.name.endswith('.xlsx')]
            
            if archivos_xlsx:
                print(f"📂 Limpiando {len(archivos_xlsx)} archivos en {ruta}")
                for blob in archivos_xlsx:
                    try:
                        blob.delete()
                        print(f"🗑️ Eliminado: {blob.name.split('/')[-1]}")
                    except Exception as e:
                        print(f"❌ Error eliminando {blob.name}: {e}")
            else:
                print(f"📂 No hay archivos .xlsx en {ruta}")
        
        print("✅ Limpieza del bucket completada")
        return True
        
    except Exception as e:
        print(f"❌ Error limpiando el bucket: {e}")
        return False
    
def subir_archivos_al_bucket():
    """
    Función para subir los archivos descargados al bucket de Google Cloud Storage
    """
    try:
        # Configurar las credenciales
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
        
        # Inicializar el cliente de Storage
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Buscar los archivos descargados
        archivos_excel = glob.glob(os.path.join(download_path, "Qlik Sense - Reporte en línea*.xlsx"))
        
        if not archivos_excel:
            print("❌ No se encontraron archivos para subir")
            return False
        
        print(f"📁 Encontrados {len(archivos_excel)} archivos para subir")
        
        # Procesar cada archivo
        for archivo_local in archivos_excel:
            nombre_archivo = os.path.basename(archivo_local)
            print(f"\n📤 Procesando: {nombre_archivo}")
            
            # Determinar la ruta de destino según el tipo de archivo
            if "Inventario" in nombre_archivo:
                ruta_destino = f"raw/Ventas/moderno/exito/inventario/{nombre_archivo}"
                print(f"📋 Tipo: Inventario")
            elif "Ventas Mensuales" in nombre_archivo:
                ruta_destino = f"raw/Ventas/moderno/exito/ventas/{nombre_archivo}"
                print(f"💰 Tipo: Ventas Mensuales")
            else:
                print(f"⚠️ Tipo de archivo no reconocido: {nombre_archivo}")
                continue
            
            # Crear el blob y subir el archivo
            blob = bucket.blob(ruta_destino)
            
            try:
                print(f"⬆️ Subiendo a: gs://{bucket_name}/{ruta_destino}")
                blob.upload_from_filename(archivo_local)
                
                # Verificar que se subió correctamente
                if blob.exists():
                    print(f"✅ Archivo subido exitosamente")
                    print(f"📊 Tamaño: {blob.size} bytes")
                    print(f"🕐 Fecha de subida: {blob.time_created}")
                else:
                    print(f"❌ Error: El archivo no se encuentra en el bucket")
                    
            except Exception as upload_error:
                print(f"❌ Error subiendo {nombre_archivo}: {upload_error}")
                continue
        
        print(f"\n🎉 Proceso de subida completado")
        return True
        
    except Exception as e:
        print(f"❌ Error crítico en la subida al bucket: {e}")
        return False

def limpiar_archivos_locales():
    """
    Función opcional para limpiar los archivos locales después de subirlos
    """
    try:
        archivos_excel = glob.glob(os.path.join(download_path, "Qlik Sense - Reporte en línea*.xlsx"))
        
        for archivo in archivos_excel:
            try:
                os.remove(archivo)
                print(f"🗑️ Archivo local eliminado: {os.path.basename(archivo)}")
            except Exception as e:
                print(f"⚠️ No se pudo eliminar {archivo}: {e}")
                
    except Exception as e:
        print(f"❌ Error limpiando archivos locales: {e}")

def verificar_archivos_en_bucket():
    """
    Función para verificar que los archivos están en el bucket
    """
    try:
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Verificar archivos en inventario
        inventario_blobs = list(bucket.list_blobs(prefix="raw/Ventas/moderno/exito/inventario/"))
        ventas_blobs = list(bucket.list_blobs(prefix="raw/Ventas/moderno/exito/ventas/"))
        
        print("\n📋 Archivos en inventario:")
        for blob in inventario_blobs:
            if blob.name.endswith('.xlsx'):
                print(f"  📄 {blob.name.split('/')[-1]} - {blob.size} bytes")
        
        print("\n💰 Archivos en ventas:")
        for blob in ventas_blobs:
            if blob.name.endswith('.xlsx'):
                print(f"  📄 {blob.name.split('/')[-1]} - {blob.size} bytes")
                
    except Exception as e:
        print(f"❌ Error verificando archivos en bucket: {e}")

# ============================================================================
# EJECUCIÓN PRINCIPAL - Subir archivos al bucket
# ============================================================================

if __name__ == "__main__":
    print("🚀 INICIANDO SUBIDA DE ARCHIVOS AL BUCKET")
    print("=" * 50)
    
    # Subir archivos
    if limpiar_bucket():
        subir_archivos_al_bucket()
        print("\n🔍 Verificando archivos en el bucket...")
        verificar_archivos_en_bucket()
        
        # Preguntar si quiere limpiar archivos locales
        print("\n🤔 ¿Deseas eliminar los archivos locales? (y/n)")
        # En un notebook, puedes comentar esta línea y descomentar la siguiente
        # respuesta = input().lower()
        respuesta = "n"  # Cambia a "y" si quieres eliminar automáticamente
        
        if respuesta == "y":
            limpiar_archivos_locales()
        else:
            print("📁 Archivos locales conservados")
    
    print("\n✅ PROCESO COMPLETADO")

🚀 INICIANDO SUBIDA DE ARCHIVOS AL BUCKET
🧹 Limpiando archivos existentes en el bucket...
📂 Limpiando 1 archivos en raw/Ventas/moderno/exito/inventario/
🗑️ Eliminado: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Inventario.xlsx
📂 Limpiando 1 archivos en raw/Ventas/moderno/exito/ventas/
🗑️ Eliminado: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Ventas Mensuales.xlsx
✅ Limpieza del bucket completada
📁 Encontrados 2 archivos para subir

📤 Procesando: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Inventario.xlsx
📋 Tipo: Inventario
⬆️ Subiendo a: gs://bucket-quickstart_croc_830/raw/Ventas/moderno/exito/inventario/Qlik Sense - Reporte en línea - 26 de agosto de 2025 Inventario.xlsx
✅ Archivo subido exitosamente
📊 Tamaño: 1111177 bytes
🕐 Fecha de subida: 2025-08-26 21:59:06.110000+00:00

📤 Procesando: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Ventas Mensuales.xlsx
💰 Tipo: Ventas Mensuales
⬆️ Subiendo a: gs://bucket-quickstart_croc_830/raw/Ventas/moderno/exito/

In [ ]:
limpiar_archivos_locales()

🗑️ Archivo local eliminado: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Inventario.xlsx
🗑️ Archivo local eliminado: Qlik Sense - Reporte en línea - 26 de agosto de 2025 Ventas Mensuales.xlsx
